In [1]:
import os
import glob
import sys


import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession



conf = SparkConf().setAppName("TestName")

In [2]:
sc = SparkContext().getOrCreate(conf)

In [3]:
input_rdd = sc.textFile("///movies.csv")

In [ ]:
input_rdd.

In [11]:
input_rdd.first()

'movieId,title,genres'

In [12]:
input_list = input_rdd.map(lambda line: line.split(','))

In [13]:
input_list.first()

['movieId', 'title', 'genres']

In [14]:
input_rdd.take(4)

['movieId,title,genres',
 '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 '2,Jumanji (1995),Adventure|Children|Fantasy',
 '3,Grumpier Old Men (1995),Comedy|Romance']

In [15]:
movie_info_rdd = input_rdd.filter(lambda line: 'movieId' not in line)

In [16]:
movie_info_rdd.first()

'1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy'

In [17]:
movie_info_list_rdd = movie_info_rdd.map(lambda x: x.split(','))

In [18]:
movie_info_list_rdd.first()[-1]

'Adventure|Animation|Children|Comedy|Fantasy'

In [19]:
movie_cat_rdd = movie_info_list_rdd.flatMap(lambda x: x[-1].split('|'))
movie_cat_rdd.take(10)

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Adventure',
 'Children',
 'Fantasy',
 'Comedy',
 'Romance']

In [20]:
cat_count = movie_cat_rdd.countByValue()
sorted(cat_count.items(), key=lambda k_v: k_v[1], reverse=True)

[('Drama', 4365),
 ('Comedy', 3315),
 ('Thriller', 1729),
 ('Romance', 1545),
 ('Action', 1545),
 ('Adventure', 1117),
 ('Crime', 1100),
 ('Horror', 877),
 ('Sci-Fi', 792),
 ('Fantasy', 654),
 ('Children', 583),
 ('Mystery', 543),
 ('Documentary', 495),
 ('Animation', 447),
 ('Musical', 394),
 ('War', 367),
 ('Western', 168),
 ('IMAX', 153),
 ('Film-Noir', 133),
 ('(no genres listed)', 18)]

In [22]:
import re
def get_year(name):
    year = None
    try:
        pattern = re.compile(r"\((\d{4})\)")
        year = int(pattern.findall(name)[0])
    except ValueError:
        pass
    except IndexError:
        pass
    
    return year

In [23]:
movie_year_rdd = movie_info_list_rdd.map(lambda x: get_year(x[1]))

In [24]:
movie_year_rdd.filter(lambda x: x is not None).min()

1916

In [25]:
movie_year_rdd.filter(lambda x: x is not None).filter(lambda x: x  > 1000).min()

1916

In [26]:
movie_year_rdd.filter(lambda x: x is not None).max()

2016

In [27]:
sc.stop()